In [10]:
import numpy as np
import h5py
import pandas as pd

hf_x = h5py.File('../data/logfbank_x.h5', 'r')
x_train = hf_x.get('x_train_data')[()]

hf_y = h5py.File('../data/logfbank_y.h5', 'r')
y_train = hf_y.get('y_train_data')[()]



In [5]:
(x_train.shape, y_train.shape)

((64727, 49, 26), (64727, 12))

In [16]:
G = pd.read_csv('../data/group.csv')
G = np.array(G.values)

In [17]:
from tensorflow.keras.layers import GRU, GlobalMaxPooling2D, Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, Conv3D, ConvLSTM2D
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split

In [18]:
gru = Sequential()
#model.add(GRU(256,input_shape=(99,26)))
gru.add(GRU(256,input_shape=(49,26)))
gru.add(Dropout(0.5))
gru.add(Dense(12, activation='softmax'))
#model.add(Dense(12, activation='sigmoid'))
gru.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
gru.summary()
weights = gru.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_gru_2 (UnifiedGRU)   (None, 256)               218112    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3084      
Total params: 221,196
Trainable params: 221,196
Non-trainable params: 0
_________________________________________________________________


In [ ]:
NUM_FOLDS = 4
EPOCHS = 10
BATCH_SIZE = 16
BAGS =1

kf = GroupKFold(n_splits=NUM_FOLDS)

for bag in range(BAGS):
    fold = 0

    val_loss = np.ones((EPOCHS,NUM_FOLDS),np.float32)

    for train, val in kf.split(x_train,y_train,G):
        gru.set_weights(weights)
        gru.reset_states()
        history = gru.fit(x_train[train], y_train[train], batch_size=BATCH_SIZE, validation_data=(x_train[val], y_train[val]), epochs=EPOCHS, shuffle=True, verbose=1)
        val_loss[:,fold] = history.history['val_loss']
        fold += 1

    val_mean = np.mean(val_loss,axis=1)
    best_loss = np.min(val_mean)
    best_epoch = np.argmin(val_mean)
    print('Best epoch: {} Best loss: {}'.format(best_epoch,best_loss))
    
    gru.set_weights(weights)
    gru.reset_states()
    gru.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=best_epoch, shuffle=True, verbose=1)

    #model.save('models/gru_{}_{}.h5'.format(bag+1,best_loss))

Train on 48545 samples, validate on 16182 samples
Epoch 1/10
48545/48545 [==============================] - 162s 3ms/sample - loss: 0.5987 - categorical_accuracy: 0.8169 - val_loss: 0.3054 - val_categorical_accuracy: 0.9043
Epoch 2/10
48545/48545 [==============================] - 160s 3ms/sample - loss: 0.3208 - categorical_accuracy: 0.9032 - val_loss: 0.2889 - val_categorical_accuracy: 0.9114
Epoch 3/10
48545/48545 [==============================] - 166s 3ms/sample - loss: 0.2733 - categorical_accuracy: 0.9188 - val_loss: 0.2613 - val_categorical_accuracy: 0.9197
Epoch 4/10
48545/48545 [==============================] - 161s 3ms/sample - loss: 0.2543 - categorical_accuracy: 0.9224 - val_loss: 0.2587 - val_categorical_accuracy: 0.9216
Epoch 5/10
48545/48545 [==============================] - 162s 3ms/sample - loss: 0.2361 - categorical_accuracy: 0.9288 - val_loss: 0.2520 - val_categorical_accuracy: 0.9253
Epoch 6/10
48545/48545 [==============================] - 159s 3ms/sample - loss